In [1]:
import pandas as pd
import json
import os



In [2]:
full_df = pd.read_csv("full_cldrive.csv")
full_df.columns


Index(['kernel_path', 'num_runs', 'gsize', 'lsize', 'kernel_name', 'outcome',
       'device_name', 'work_item_local_mem_size', 'work_item_private_mem_size',
       'transferred_bytes', 'transfer_time_ns', 'kernel_time_ns', 'stderr'],
      dtype='object')

In [3]:
target_errors = ["CL_ERROR", "UNKNOWN_ERROR", "FAILED"]

failed_df = full_df[full_df["outcome"].isin(target_errors)]
failed_df["kernel_id"] = failed_df["kernel_path"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
failed_df["outcome"].value_counts()


/tmp/ipykernel_260278/2005725956.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failed_df["kernel_id"] = failed_df["kernel_path"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])


outcome
CL_ERROR         55411
UNKNOWN_ERROR     1062
FAILED             862
Name: count, dtype: int64

In [8]:
# randomly select 100 kernels that does not have the same kernel_id
result_df = failed_df.groupby("kernel_id").first().reset_index()
result_df = result_df[["kernel_path", "gsize", "lsize", "outcome"]]
# result_df = result_df.sample(n=100, random_state=42)
result_df.to_json("mem_analysis_pilot.jsonl", orient="records", lines=True)


In [11]:
BACKUP_DIR = "mem-analysis/"
for i, row in result_df.iterrows():
    mem_file = os.path.join(BACKUP_DIR, os.path.splitext(os.path.basename(row["kernel_path"]))[0] + ".json")
    if os.path.exists(mem_file):
        print(mem_file)


mem-analysis/017cf2dc3a55e48deec9a19bbcb94ac1ed1643465000cb097daac9ef0d529a7e.json
mem-analysis/02bac87c12d5d2736fdb7d7a1328abdf6e04ed0a0553da5c58202df3dc31cece.json
mem-analysis/033d87695007fa01f1d0e5e4f478299420dea48fe00089a73457336c1ce489a9.json
mem-analysis/035f57cf2b4d3f44af0cfab15a4f64bf363d94fed0d1bab20cb7095b34493674.json
mem-analysis/06a8472cb37efb559a02156fbdd32eef61cab11946793703300086e9dfcc92d1.json
mem-analysis/07a86874c21c6f3dbef03ec1be9df5bc155d5f22f92397134a218729b4a7d650.json
mem-analysis/096a36db49305f2b02f58b0972316f198e95b513c42c5949b360bb959a8bfc6d.json
mem-analysis/09bdf720615887150deb9f2c4deae45c5118c7391bc662b2c8835c34118b102e.json
mem-analysis/1016ca27a4ce50b79523f9733f984fa76e1b5da820188357d851c67789a9529d.json
mem-analysis/13bc8b8a317d0c1e5779b3100ca32ac313b55ef3a87ff1d2e56c7f18d08102d2.json
mem-analysis/1725faeda16c96df4aacd2812c319704102040e7e2f4b5b8caf31f87832cb170.json
mem-analysis/1870a99aad90ca925bb9b77cdeca1c7ece030c6565f947bc5d4f607e0762d4a3.json
mem-

In [4]:
# read all files in the directory, each file is a json, to a list, then create a dataframe
BACKUP_DIR = "mem-analysis/"
MEM_ANALYSIS_KERNEL_DIR = "kernels-modified/"

def read_json_files(path):
    json_files = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
    jsons_data = []
    for index, js in enumerate(json_files):
        with open(os.path.join(path, js)) as json_file:
            json_text = json.load(json_file)
            jsons_data.append({"kernel_id": os.path.splitext(os.path.basename(js))[0], "mem_info": json_text, })
    return jsons_data

jsons_data = read_json_files(BACKUP_DIR)
mem_analysis_df = pd.DataFrame(jsons_data)
mem_analysis_df["kernel_path"] = mem_analysis_df["kernel_id"].apply(lambda x: os.path.join(MEM_ANALYSIS_KERNEL_DIR, x + ".cl"))
mem_analysis_df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'backup/'

In [ ]:
joined_df = pd.merge(failed_df, mem_analysis_df, on="kernel_id", how="inner")
joined_df.columns


Index(['kernel_path_x', 'num_runs', 'gsize', 'lsize', 'kernel_name', 'outcome',
       'device_name', 'work_item_local_mem_size', 'work_item_private_mem_size',
       'transferred_bytes', 'transfer_time_ns', 'kernel_time_ns', 'stderr',
       'kernel_id', 'mem_info', 'kernel_path_y'],
      dtype='object')

In [ ]:
# randomly select 100 kernels that does not have the same kernel_id
result_df = joined_df.groupby("kernel_id").first().reset_index()
result_df["kernel_path"] = result_df["kernel_path_y"]
result_df = result_df[["kernel_path", "kernel_id", "gsize", "lsize", "mem_info", "outcome"]]
result_df.to_json("mem_analysis_pilot.jsonl", orient="records", lines=True)
